In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import datetime as dt
import os

"""
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
os.environ['CUDA_VISIBLE_DEVICES']='0'
"""
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
print(tf.__version__)
print(gpus)

import tensorflow as tf
from tensorflow.keras import * 
import time

tf.config.set_soft_device_placement(True)
tf.debugging.set_log_device_placement(True)

gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

t=time.time()
with tf.device("/gpu:0"):
    tf.random.set_seed(0)
    a = tf.random.uniform((10000,10000),minval = 0,maxval = 3.0)
    c = tf.matmul(a, tf.transpose(a))
    d = tf.reduce_sum(c)
print('gpu: ', time.time()-t)

t=time.time()
with tf.device("/cpu:0"):
    tf.random.set_seed(0)
    a = tf.random.uniform((10000,10000),minval = 0,maxval = 3.0)
    c = tf.matmul(a, tf.transpose(a))
    d = tf.reduce_sum(c)
print('cpu: ', time.time()-t)

In [ ]:
import os
import cv2 
import numpy as np
from PIL import Image
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import cv2
%matplotlib inline

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
import cv2
%matplotlib inline

##定義ImageDataGenerator
train_generator = ImageDataGenerator( featurewise_center=True,
                             featurewise_std_normalization=True,
                             rotation_range=60,
                             width_shift_range=0.3,
                            height_shift_range=0.3,
                             shear_range=0.3,
                             zoom_range=0.3,
                             horizontal_flip=True,
                             vertical_flip=True,dtype=np.float32)

train_generator = train_generator.flow_from_directory(
        r'C:\Users\LAB543\Desktop\DATA\opencv_test\Train_Image\C',
        target_size=(512, 512),    #將圖片大小轉化為512x512
        batch_size=31,              #每一次處理32張圖片
        class_mode='categorical',  #對類別進行獨熱編碼
        save_to_dir=r'C:\Users\LAB543\Desktop\DATA\opencv_test\train_augmentation\C', #將augmentation之後的圖片保存
        save_format='JPG',
        save_prefix="augmentation_")

count=1
for x_batch,y_batch in train_generator:
    print(F"------------開始第{count}次反覆運算-----------------------------")
    print(F"------------x_batch、y_batch的形狀如下----------------------")
    print(np.shape(x_batch),np.shape(y_batch))
#     print('-------------y_batch列印結果如下-----------------------------')
#     print(y_batch)
    print('============================================================')
    count+=1
    if count>200:
        break


## 以下為檔案搬移
- 將資料夾格式改為
Train_Image  Test_Image 

```
Train_Image/
...A/
......07999.jpg
......07998.jpg
...B/
......XXXXX.jpg
......XXXXX.jpg
...C/
......CCCCC.jpg
......CCCCC.jpg
```

In [ ]:
import os  				#os是用來切換路徑和創建資料夾的。
from shutil import copy #shutil 是用來複製黏貼檔的

train_dir = r'C:\Users\lab543\Desktop\DATA\Train_Image'
train_label = pd.read_csv(train_dir+ r'\..\train.csv',header = 0)
train_label_list=train_label[train_label['label']=='A'].iloc[:,0].values.tolist()
    
    
file_path = r'C:\Users\lab543\Desktop\DATA\Train_Image' #想拆分的資料夾所在路徑,也就是一大堆檔所在的路徑
save_dir = r'C:\Users\LAB543\Desktop\DATA\opencv_test\Train_Image' #save_dir 是想把複製出來的檔存放在的路徑
dir_name = "C" # A B C
 
# 獲取 file_path 下的文件和資料夾清單
# 因為 file_path 裡面沒有資料夾，所以不處理有資料夾的情況
pathDir = train_label[train_label['label']==dir_name].iloc[:,0].values.tolist() #os.listdir(file_path) 是獲取指定路徑下包含的檔或資料夾清單

for filename in pathDir: #遍歷pathDir下的所有檔filename
    print(filename)
    from_path = os.path.join(file_path, filename) #舊檔的絕對路徑(包含檔的尾碼名)
    to_path = save_dir + "\\" + dir_name          #新檔的絕對路徑
    if not os.path.isdir(to_path):# 如果 to_path 目錄不存在，則創建
        os.makedirs(to_path)
    copy(from_path, to_path)


# STAR train

In [1]:
# Ho
import os
import cv2 
import numpy as np
from PIL import Image
import pandas as pd

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from tensorflow.keras import layers,models,optimizers


from sklearn.model_selection import train_test_split

# 可以不用此方法 ， 以image_dataset_from_directory代替
# img_resize=(224,224)
# def ReFileName(dirPath):
#     mango=[]
#     for file in os.listdir(dirPath):
#         if os.path.isfile(os.path.join(dirPath, file)) == True:
#            c= os.path.basename(file)
#            name = dirPath + '\\' + c
#            img = cv2.imread(name)
#            img=cv2.resize(img,img_resize)
#            mango.append(img)
#     return mango

## image_dataset_from_directory 讀取 train_dataset、 val_dataset

- labels="inferred" （標籤是從目錄結構生成的） 
- label_mode="categorical" (對應 One Hat 格式)
- class_names=None 按照順序 A->0 B->1 C->2
- subset= training,validation, 讀取資料集，是屬於 training、validation
- seed=0  training、validation 務必一樣

In [ ]:
# train_dir=r'C:\Users\LAB543\Desktop\DATA\opencv_test\train_augmentation'
train_dataset=tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=64,
    image_size=(224, 224),
    shuffle=True,
    validation_split=0.2,
    subset='training',
    seed=0,
    interpolation="bilinear",
    follow_links=False,
)
val_dataset=tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=64,
    image_size=(224, 224),
    shuffle=True,
    validation_split=0.2,
    subset='validation',
    seed=0,
    interpolation="bilinear",
    follow_links=False,
)

## Min Max normalization

In [4]:
from tensorflow.keras import layers
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), tf.cast(y, dtype=tf.int32)))
val_dataset=val_dataset.map(lambda x, y: (normalization_layer(x), tf.cast(y, dtype=tf.int32)))

# 建立模型

In [5]:
# Ho

from tensorflow.keras.applications.vgg16 import VGG16
def create_model():
    
    net = VGG16(include_top=False, weights='imagenet', input_tensor=None,
#           input_shape = (train_data.shape[1],train_data.shape[2],train_data.shape[3])
            input_shape = (224,224, 3) 
          )
    l2 = tf.keras.regularizers.l2(0.01)

    # 透過layer_name 節取 model的某一層 block3_pool block4_conv3
    layer_name='block4_pool'

    # 設定取到該層的都使用預設參數，不訓練參數
    layer_index=net.layers.index(net.get_layer(layer_name))

    # 建立模型
    base_model = net.get_layer(layer_name).output
    base_model = layers.Conv2D(128, (1, 1), padding='valid')(base_model)
    base_model = layers.MaxPooling2D( pool_size = (3, 3) )(base_model)

    # 攤平 feature map 
    base_model = layers.Flatten()(base_model)


    base_model = layers.Dropout(0.3)(base_model)
    base_model = layers.BatchNormalization()(base_model)
    base_model = layers.Dense(units = 256, kernel_initializer = tf.keras.initializers.GlorotNormal()
             ,kernel_regularizer= l2 , activation= 'relu')(base_model)


    base_model = layers.Dense(units = 256, kernel_initializer = tf.keras.initializers.GlorotNormal()
             ,kernel_regularizer = l2 , activation= 'relu')(base_model)

    base_model = layers.Dropout(0.3)(base_model)
    output_layer = layers.Dense(3, activation='softmax', name='softmax')(base_model)

    # 設定凍結與要進行訓練的網路層
    net_final = tf.keras.models.Model(inputs=net.input, outputs=output_layer)

    for layer in net_final.layers[:layer_index]:
        layer.trainable = False

    return net_final

test_model=create_model()
test_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

#  compile fit  callbacks 設定

In [ ]:
# Ho

# 
# Generate batches of tensor image data with real-time data augmentation.

train_datagen=tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=15 , 
                             width_shift_range=0.2 , 
                             height_shift_range=0.2 ,
                             shear_range=0.2 ,
                             zoom_range=0.2 , 
                             data_format='channels_last')

callbacks_list=[
        tf.keras.callbacks.EarlyStopping(
          monitor='val_loss',mode='min', patience=18 ,restore_best_weights=True
        ),
        tf.keras.callbacks.ModelCheckpoint(
            filepath='C:/Users/lab543/Desktop/DATA/Mango_VGG16_02',
            monitor='val_loss',
            save_best_only=True,
        ),
        tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy',
                             patience=3,
                             # 3 epochs 內acc沒下降就要調整LR
                             verbose=1,
                             factor=0.5,
                             # LR降為0.5
                             min_lr=0.00001
                             # 最小 LR 到0.00001就不再下降
                             )]

precision=tf.keras.metrics.Precision(name='precision')
recall=tf.keras.metrics.Recall(name='recall')
accuracy=tf.keras.metrics.CategoricalAccuracy(name='accuracy')


test_model.compile(optimizer='Adam' , loss='categorical_crossentropy' , metrics=[precision,recall,accuracy])
history=test_model.fit(train_dataset,
                       validation_data=val_dataset,
                       epochs=100,
                        callbacks=callbacks_list
                      )

# history=test_model.fit(train_datagen.flow(train_data,train_label,batch_size=128),
#                        epochs=30,
#                        validation_data=(val_data,val_label),
#                        callbacks=callbacks_list)


Epoch 1/100
56/56 [==============================] - 32s 460ms/step - loss: 7.6045 - precision: 0.5026 - recall: 0.3833 - accuracy: 0.4778 - val_loss: 5.4425 - val_precision: 0.6224 - val_recall: 0.5824 - val_accuracy: 0.6036
INFO:tensorflow:Assets written to: C:/Users/lab543/Desktop/DATA\Mango_VGG16_02\assets
Epoch 2/100
56/56 [==============================] - 23s 397ms/step - loss: 4.7224 - precision: 0.7248 - recall: 0.5387 - accuracy: 0.6447 - val_loss: 3.1685 - val_precision: 0.8106 - val_recall: 0.6804 - val_accuracy: 0.7520
INFO:tensorflow:Assets written to: C:/Users/lab543/Desktop/DATA\Mango_VGG16_02\assets
Epoch 3/100
56/56 [==============================] - 23s 401ms/step - loss: 2.9181 - precision: 0.7618 - recall: 0.6363 - accuracy: 0.7164 - val_loss: 1.9971 - val_precision: 0.8540 - val_recall: 0.7726 - val_accuracy: 0.8128
INFO:tensorflow:Assets written to: C:/Users/lab543/Desktop/DATA\Mango_VGG16_02\assets
Epoch 4/100
56/56 [==============================] - 24s 404ms/s

#   畫 train 圖  、評估測試

In [ ]:
# HO
import matplotlib.pyplot as plot  
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import datetime as dt
import os
def train_history_graphic( history       # 資料集合
              , history_key1  # 資料集合裡面的來源 1 (有 loss, acc, val_loss, val_acc 四種)
              , history_key2  # 資料集合裡面的來源 2 (有 loss, acc, val_loss, val_acc 四種)
              , y_label       # Y 軸標籤文字
                         ) :
    # 資料來源 1
    plot.plot( history.history[history_key1] )
    # 資料來源 2
    plot.plot( history.history[history_key2] )
    # 標題
    plot.title( 'train history' )

    # X 軸標籤文字
    plot.xlabel( 'epochs' )

    # Y 軸標籤文字
    plot.ylabel( y_label )

    # 設定圖例
    # (參數 1 為圖例說明, 有幾個資料來源, 就對應幾個圖例說明)
    # (參數 2 為圖例位置, upper 為上面, lower 為下面, left 為左邊, right 為右邊)
    plot.legend( ['train', 'validate']
               , loc = 'upper left'
               )

    # 顯示畫布
    plot.show()

train_history_graphic( history, 'loss', 'val_loss', 'loss' )
train_history_graphic( history, 'accuracy', 'val_accuracy', 'accuracy' )
train_history_graphic( history, 'precision', 'val_precision', 'precision' )
train_history_graphic( history, 'recall', 'val_recall', 'recall' )


In [ ]:
# 顯示 Model 評估測試資料的

test_dir=r'D:\_YUN\24_ML\Work02\Mango\DATA\Test_Image_A_B_C'
test_dataset=tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=64,
    image_size=(224, 224),
    shuffle=True,
    interpolation="bilinear"
)

from tensorflow.keras import layers
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
test_dataset = test_dataset.map(lambda x, y: (normalization_layer(x), tf.cast(y, dtype=tf.int32)))

# 顯示 Model 評估測試資料的
preds=test_model.evaluate(test_dataset)

print('loss ： %.5f' % preds[0])
print('precision：%.5f' % preds[1])
print('recall：%.5f' % preds[2])
print('accuracy： %.5f' % preds[3])
F1 = 2 * (preds[3] * preds[2]) / (preds[3] + preds[2])
print('f1score：%.5f' % F1)

# load_model 預測

In [2]:
# 2020-05-01 D:\_YUN\24_ML\Work02\Mango\ModelDATA\Mango_VGG16_03
# 2021-04-23 load_model
# 
test_model = keras.models.load_model(r'D:\_YUN\24_ML\Work02\Mango\ModelDATA\Mango_VGG16_03')

In [12]:
test_dir=r'D:\_YUN\24_ML\Work02\Mango\DATA\Test_Image_A_B_C'
test_dataset=tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=64,
    image_size=(224, 224),
    shuffle=True,
    interpolation="bilinear"
)

from tensorflow.keras import layers
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
test_dataset = test_dataset.map(lambda x, y: (normalization_layer(x), tf.cast(y, dtype=tf.int32)))

Found 250 files belonging to 3 classes.


In [13]:
# 顯示 Model 評估測試資料的
preds=test_model.evaluate(test_dataset)

print('loss ： %.5f' % preds[0])
print('precision：%.5f' % preds[1])
print('recall：%.5f' % preds[2])
print('accuracy： %.5f' % preds[3])
F1 = 2 * (preds[3] * preds[2]) / (preds[3] + preds[2])
print('f1score：%.5f' % F1)

4/4 [==============================] - 41s 9s/step - loss: 0.7813 - precision: 0.7489 - recall: 0.7040 - accuracy: 0.7360
loss ： 0.78128
precision：0.74894
recall：0.70400
accuracy： 0.73600
f1score：0.71964
